In [1]:
# retrieve plasmid sequence from GenBank
from pydna.genbank import Genbank
from Bio import SeqIO
from Bio.Restriction import AllEnzymes
from pydna.parsers import parse_snapgene
import matplotlib.pyplot as plt
import numpy as np
import itertools

from snapgene_reader import snapgene_file_to_dict, snapgene_file_to_seqrecord

In [2]:
AllEnzymes.get("EcoRI")

EcoRI

In [3]:
from pydna.dseqrecord import Dseqrecord
from pydna.design import primer_design

In [4]:
def load_from_genebank(email, sequence):
    gb = Genbank(users_email=email)  
    seq = gb.nucleotide(sequence)   
    return seq

In [5]:
email = "mikelgarciadata@gmail.com"

In [6]:
# Some gene sequence
seqs_id = ["M61151.1", "X51424.1", "PV006683.1", "PV006682.1", "OR455449.1"]
# Create a Dseq object
seqs = []
for seq in seqs_id:
    seqs.append(load_from_genebank(email, seq))


In [8]:
print(type(seqs[0]))

<class 'pydna.genbankrecord.GenbankRecord'>


In [6]:
# Define primer length (typical range 18-22 nt)
primer_length = 20

In [12]:
# Use Primer3 for more sophisticated design (optional):
primers = list(map(lambda seq: primer_design(seq, target_tm=55, limit=20), seqs))

for primer in primers:
    print("Forward primer:", primer.forward_primer.seq)
    print("Reverse primer:", primer.reverse_primer.seq)

Forward primer: GTCGACAGTCGCAACAGCAA
Reverse primer: GTCGACGTTTGCCGGCGAGG
Forward primer: GATCGATTACAAACAATTGAGAA
Reverse primer: GCAGATGTTGCTAAAAGTAATT
Forward primer: GTGTCGGTCATCGAATCCAC
Reverse primer: TCAGCGCGCGCCGGACAGAT
Forward primer: ATGGCGCACGGGCTGGGTGG
Reverse primer: TTATAAGTCGACATGCGCGTCTCCTTGCTTCGCTGG
Forward primer: ATGTGTTGCGAAGATCACCT
Reverse primer: TCATCCTGTCATGTCCCTGC


In [23]:
enzyme_name = "EcoRI"

enz_seq = AllEnzymes.get(enzyme_name).site.lower()
for seq in seqs:
    amplicon = primer_design(seq, target_tm=60, limit = 20)
    fwd_primer, rvs_primer = amplicon.primers()
    fwd_primer_enzyme = "tt" + enz_seq + fwd_primer
    rvs_primer_enzyme = "tt" + enz_seq + rvs_primer
    print(f"Forward primer: {fwd_primer_enzyme.seq}")
    print(f"Reverse primer: {rvs_primer_enzyme.seq}")

Forward primer: ttgaattcGTCGACAGTCGCAACAGCAA
Reverse primer: ttgaattcGTCGACGTTTGCCGGCGAGG
Forward primer: ttgaattcGATCGATTACAAACAATTGAGAACGA
Reverse primer: ttgaattcGCAGATGTTGCTAAAAGTAATTTCTTAAAC
Forward primer: ttgaattcGTGTCGGTCATCGAATCCAC
Reverse primer: ttgaattcTCAGCGCGCGCCGGACAGAT
Forward primer: ttgaattcATGGCGCACGGGCTGGGTGG
Reverse primer: ttgaattcTTATAAGTCGACATGCGCGTCTCCTTGCTTCGCTGG
Forward primer: ttgaattcATGTGTTGCGAAGATCACCT
Reverse primer: ttgaattcTCATCCTGTCATGTCCCTGC


In [19]:
def create_primers(seq, enzyme, target_tm, limit):
    amplicon = primer_design(seq, target_tm=target_tm, limit = target_tm)
    fwd_primer, rvs_primer = amplicon.primers()
    fwd_primer_enzyme = "tt" + enz_seq + fwd_primer
    rvs_primer_enzyme = "tt" + enz_seq + rvs_primer

    return fwd_primer_enzyme, rvs_primer_enzyme

In [24]:
enzyme_name = "EcoRI"

enz_seq = AllEnzymes.get(enzyme_name).site.lower()
primers = list(map(lambda seq: create_primers(seq, enz_seq, target_tm=60, limit = 20), seqs))
for fwd_primer_enzyme, rvs_primer_enzyme in primers:
    print(f"Forward primer: {fwd_primer_enzyme.seq}")
    print(f"Reverse primer: {rvs_primer_enzyme.seq}")

Forward primer: ttgaattcGTCGACAGTCGCAACAGCAATCGAGGGGTGTTGATCAACCTTGGCCAGTTGCCCTTCGTC
Reverse primer: ttgaattcGTCGACGTTTGCCGGCGAGGGCATCGTCTTTCCTCGATTTGAAATTGGGGCCACCGGTGT
Forward primer: ttgaattcGATCGATTACAAACAATTGAGAACGAAAAAGAACAGAGTAAGAATAACGCAGACAAAGCT
Reverse primer: ttgaattcGCAGATGTTGCTAAAAGTAATTTCTTAAACATGAAATCAATCCCTTTCTTTTAAATTATTTACATGTAAATGATAACATTATGTTCAGACTAGAAAAATAATACTTTTTACTCAAATATTAGAATAATTTTGAACTATCTGTAACAAAATAAACATAAGTATCTCCTTG
Forward primer: ttgaattcGTGTCGGTCATCGAATCCACTGACCGAACGAGGCAAGCCGACATGACGCGACACGACATC
Reverse primer: ttgaattcTCAGCGCGCGCCGGACAGATGCAGCGTGAACCACTCCACCGCCACCCGGCTCGTTTCGTC
Forward primer: ttgaattcATGGCGCACGGGCTGGGTGGAATCAAGGAAATGCGGCTCGATGCCTATGCGCAACGCTTT
Reverse primer: ttgaattcTTATAAGTCGACATGCGCGTCTCCTTGCTTCGCTGGAACGTGACGGTGCAGAAATGCCAGCTGGTCGGCGACCAC
Forward primer: ttgaattcATGTGTTGCGAAGATCACCTGGTCCCCCCACAAATCCAAAAAATGATTGCGGCCTACCAA
Reverse primer: ttgaattcTCATCCTGTCATGTCCCTGCGCGTTGCTATCGACCGATACTTGTAACGTGCTTGGTCGAG


In [ ]:
columns = ["Gene", "Forward Primer", "Reverse Primer"]